In [27]:
import numpy as np
from sklearn.decomposition import PCA
import h5py
import pickle
import matplotlib.pyplot as plt
import GPy

In [28]:
def gp_predict(model, params):
    """
    Predicts the weights matrix to feed inverse PCA from physical parameters.

    Input :
    - model : GP model
    - params : physical parameters (flux, radius, shear profile, psf fwhm)

    Output :
    - predic[0] : predicted weights
    """
    predict = model.predict(params)
    return predict[0]


def emulator(pca_model, gp_model, params):
    """
    Emulates new images from physical parameters.

    Input :
    - pca : PCA model
    - gp_model : GP model
    - params : physical parameters (flux, radius, shear profile, psf fwhm)

    Output :
    - reconstructed : Emulated image
    """
    # Weights prediction
    params = np.expand_dims(params, axis = 0)
    pred_weights = gp_predict(gp_model, params)

    # Inverse PCA (pred_weights * basis + mean)
    reconstructed = pca_model.inverse_transform(pred_weights)
    return reconstructed[0]


In [29]:
save_loc = "/Users/nramachandra/Desktop/Projects/AGNfinder/FSPS/"

with h5py.File(save_loc + 'fsps_cache.hdf5', 'r') as f:
   X_loaded = f['fsps_cache']['X'][...]
   Y_loaded = f['fsps_cache']['Y'][...] 


In [30]:
## logging and clipping
# X_loaded = np.log10(X_loaded)
Y_loaded = np.log10(Y_loaded[:, 100:])

# ### rescaling 
y_mean = np.mean(Y_loaded, axis=0)
y_mult = np.max(Y_loaded - y_mean, axis=0)

y_train = (Y_loaded - y_mean)/y_mult

# ### rescaling 
x_mean = np.mean(X_loaded, axis=0)
x_mult = np.max(X_loaded - x_mean, axis=0)

x_train = (X_loaded - x_mean)/x_mult

## loading trained GP and PCA models

In [32]:
num_params = 3
num_bases = 10

with open('pcaModel.pickle', 'rb') as handle:
    pca_model = pickle.load(handle)

gp_model = GPy.models.GPRegression.load_model('gpfit_'+str(num_bases)+'_'+str(num_params) + '.zip')

In [ ]:
from matplotlib import gridspec
from itertools import cycle
import matplotlib.ticker as ticker

plt.rc('text', usetex=True)  # Slower
plt.rc('font', size=18)  # 18 usually

plt.figure(999, figsize=(14, 12))


gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1])
gs.update(hspace=0.1, left=0.2, bottom=0.15, wspace=0.25)
ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1])

# ax0.set_ylabel(r'$P(k)$',  fontsize = 18)

# ax1.set_xlabel(r'$k$',  fontsize = 18)
ax1.axhline(y=0, ls='dashed')


ax0.set_yscale('log')
ax0.set_xscale('log')
ax1.set_xscale('log')

ax0.plot(10**(  (y_train*y_mult) + y_mean ), 'k-', alpha = 0.01)

ax1.set_ylabel(r'emu/test - 1',  fontsize = 18)


start, end = ax0.get_ylim()
ax0.yaxis.set_major_formatter(ticker.FormatStrFormatter('%0.1f'))

color_id = 0
for i in range(10):
    color_id += 1
    rand_idx = (np.random.randint(100, 2000))
    rand_params = x_train[rand_idx]
    y_emu = emulator(pca_model, gp_model, rand_params)
    y_true = y_train[rand_idx]
    ax0.plot(10**(  (y_emu.T*y_mult) + y_mean ), color=plt.cm.Set1(color_id), alpha = 0.5)
    ax0.plot(10**( (y_true.T*y_mult) + y_mean ), '--', color=plt.cm.Set1(color_id))

    ax1.plot((10**( (y_emu.T*y_mult) + y_mean) )/(10**( (y_true.T*y_mult) + y_mean) ) - 1, color=plt.cm.Set1(color_id))

    
plt.show()

In [34]:
x_train[23]

array([0.67034251, 0.28784201, 0.01118953])

In [36]:
%%timeit

emulator(pca_model, gp_model, rand_params)

5.68 ms ± 1.04 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
